In [2]:
import json
import os
import os.path as osp
import sys
from PIL import Image
import pickle as pkl
import cv2

from detectron.datasets.json_dataset import JsonDataset
from detectron.utils.io import load_object, save_object
import detectron.utils.vis as vis_utils


def checkMkdir(dirname):
    if not osp.isdir(dirname):
        os.makedirs(dirname)


modelType = 'HF_Detector' # 'Fashion_Detector' #'COCO_Mask_Detector'
fixedPrec = 0.8

outThrshBoxPath = osp.join('/media/data/chnxi/GOD/threshold_bboxes/') #, 'prec_at_{}'.format(fixedPrec))
modelOutBoxFolder = osp.join(outThrshBoxPath, modelType)
checkMkdir(modelOutBoxFolder)

if modelType == 'Fashion_Detector':
    cls_thrsh_file = '/home/chnxi/ssd/backfill/data/class_thresholds_at_pr.pkl'
    boxesOnDataset = {
        #'furniture_train': '/home/chnxi/ssd/backfill/FashionV2_on_HFTrain/outputs/FashionV2_HFTrain_by_image.json',
        #'COCO_trainval': '/home/chnxi/ssd/backfill/FashionV2_on_COCOTrainval/outputs/FashionV2_COCOTrainvalBbox_by_image.json',
        #'OpenImage_train': '/home/chnxi/ssd/backfill/FashionV2_on_OpenImage/outputs/FashionV2_OpenImageTrain_Bboxes_by_image.json',
        'Object365_train': '/home/chnxi/ssd/backfill/FashionV2_on_Object365/outputs/FashionV2_Object365Train_by_image.json',
    }
elif modelType == 'HF_Detector':
    cls_thrsh_file = '/home/chnxi/Detectron/backfill/HF_Detector/class_thresholds_at_pr.pkl'
    boxFilePath = '/home/chnxi/Detectron/backfill/HF_Detector/'
    boxesOnDataset = {#'FashionV2_train': boxFilePath + 'FashionV2_train/test/FashionV2_train/retinanet/detections.pkl',
    #'OpenImage_train': boxFilePath + 'OpenImage_train/test/OpenImage_train/retinanet/detections.pkl', 
    #'coco_2014_valminusminival':  boxFilePath + 'coco_trainval/test/coco_2014_valminusminival/retinanet/detections.pkl',
    #'coco_2014_train': boxFilePath + 'coco_trainval/test/coco_2014_train/retinanet/detections.pkl',
    'Object365_train': boxFilePath + 'Object365_train/Object365_train/retinanet/detections.pkl',
                     }
    
elif modelType == 'COCO_Mask_Detector':
    cls_thrsh_file = '/home/chnxi/Detectron/backfill/COCO_Mask_Detector/class_thresholds_at_pr.pkl'
    boxFilePath = '/home/chnxi/Detectron/backfill/COCO_Mask_Detector/'
    boxesOnDataset = {'FashionV2_train': boxFilePath + 'FashionV2_train/test/FashionV2_train/generalized_rcnn/detections.pkl',
    'OpenImage_train': boxFilePath + 'OpenImage_train/test/OpenImage_train/generalized_rcnn/detections.pkl', 
    'furniture_train': boxFilePath + 'furniture_train/test/furniture_train/generalized_rcnn/detections.pkl'
                     }

    
print ("modelType = {}===================".format(modelType))
print ("cls_thrsh_file = {}===================".format(cls_thrsh_file))
print ("modelOutBoxFolder = {}===================".format(modelOutBoxFolder))

modelType = HF_Detector===================
cls_thrsh_file = /home/chnxi/Detectron/backfill/HF_Detector/class_thresholds_at_pr.pkl===================
modelOutBoxFolder = /media/data/chnxi/GOD/threshold_bboxes/HF_Detector===================


In [3]:
cls_thrsh_dict = load_object(cls_thrsh_file)
print (cls_thrsh_dict.keys())
classes = cls_thrsh_dict['classes']

apply_cls_thrsh = cls_thrsh_dict['prec_at'][fixedPrec]

print(len(apply_cls_thrsh))
print(apply_cls_thrsh)
print (len(classes))


dict_keys(['prec_at', 'rec_at', 'classes', 'cls_aps'])
58
[0.53108162 0.40394038 0.49834916 0.58538604 0.55322909 0.50788492
 0.41246161 0.46174601 0.61438024 0.589773   0.63669521 0.4913508
 0.44736999 0.49898207 0.55152893 0.5989415  0.58256692 0.60671854
 0.42294472 0.45957702 0.5237124  0.50280547 0.51417321 0.51839036
 0.55527121 0.65307546 0.50789326 0.41195145 0.49644741 0.42217097
 0.53769779 0.52608311 0.57853234 0.51026779 0.56736928 0.49196643
 0.70769304 0.74092299 0.63649017 0.52677852 0.50551462 0.45433584
 0.46677014 0.48081741 0.52257943 0.54792863 0.56537491 0.49752668
 0.44553861 0.58268529 0.46840149 0.56965488 0.62115413 0.35948369
 0.45514172 0.5713113  0.76778102 0.45063075]
58


In [ ]:
if modelType == 'Fashion_Detector':
    for dataset in boxesOnDataset:
        print ("model = {}, dataset = {}".format(modelType, dataset))
        outBoxFile = osp.join(modelOutBoxFolder, dataset + '.json')
        print ("outBoxFile = {}".format(outBoxFile))
        orgBoxFile = boxesOnDataset[dataset]
        print ("loading {}".format(orgBoxFile))
        dets = json.load(open(orgBoxFile,'r'))
        print ("loaded")
        imgNum = len(dets)
        print ("imgNum = {}".format(imgNum))
        validBoxNum = 0
        for ix, entry in enumerate(dets):
            img_boxes = entry['boxes']
            img_boxes_thrshed = []
            for box in img_boxes:
                cid = box['category_id'] -1
                #print ("category: {}, cid = {}, thrsh = {}, score = {}".format(box['category_name'], cid, thresholds[cid], box['score']))
                if box['score'] > apply_cls_thrsh[cid]:
                    img_boxes_thrshed.append(box)
            
            dets[ix]['boxes'] = img_boxes_thrshed
            if ix % 10000 == 0:
                print (dets[ix])
            validBoxNum += len(img_boxes_thrshed)
        print ("Total boxNum = {}".format(validBoxNum))
        print ("saving boxes to {}".format(outBoxFile))
        with open(outBoxFile,'w') as fout:
            json.dump(dets, fout)
        print ("Done!")

In [4]:
if modelType in ['HF_Detector', 'COCO_Mask_Detector']:
    for dataset in boxesOnDataset:
        print ("model = {}, dataset = {}".format(modelType, dataset))
        outBoxFile = osp.join(modelOutBoxFolder, dataset + '.pkl')
        print ("outBoxFile = {}".format(outBoxFile))
        orgBoxFile = boxesOnDataset[dataset]
        print ("loading {}".format(orgBoxFile))
        dets = load_object(orgBoxFile)
        print ("loaded")
        all_boxes = dets['all_boxes'][1:] # all_boxes[0] is for background
        classNum = len(all_boxes)
        imgNum = len(all_boxes[0])
        print ("classNum = {}, imgNum = {}".format(classNum, imgNum))
        dataset_thrsh_boxes = [] # in dataset_thrsh_boxes[imgid][cid] format
        validBoxNum = 0

        for imgid in range(imgNum):
            keep_img_boxes = []
            for cid in range(classNum):
                img_cls_boxes = all_boxes[cid][imgid]
                keep_img_cls_boxes = [bbox for bbox in img_cls_boxes if bbox[-1] > apply_cls_thrsh[cid]]
                validBoxNum += len(keep_img_cls_boxes)
                keep_img_boxes.append(keep_img_cls_boxes)
            dataset_thrsh_boxes.append(keep_img_boxes)

        print ("Total boxNum = {}".format(validBoxNum))
        print ("saving boxes to {}".format(outBoxFile))
        save_object(dataset_thrsh_boxes, outBoxFile)
        print ("Done!")


model = HF_Detector, dataset = Object365_train
outBoxFile = /media/data/chnxi/GOD/threshold_bboxes/HF_Detector/Object365_train.pkl
loading /home/chnxi/Detectron/backfill/HF_Detector/Object365_train/Object365_train/retinanet/detections.pkl
loaded
classNum = 58, imgNum = 608606
Total boxNum = 922904
saving boxes to /media/data/chnxi/GOD/threshold_bboxes/HF_Detector/Object365_train.pkl
Done!
